In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision
import os
import random
import matplotlib.image as img
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import glob
import gc

In [ ]:
SEED = 42
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# train_list = []
# test_list = []
# label_list = []

# for dirname, _, filenames in os.walk('../input/imagenetmini-1000/imagenet-mini/train/'):
#     label_list.append(str(dirname).split('/')[5])
# label_list = list(set(label_list))[1:]

# for dirname, _, filenames in os.walk('../input/imagenetmini-1000/imagenet-mini/train/'):
#     for filename in filenames:
#         train_list.append(str(os.path.join(dirname, filename)))
        
# for dirname, _, filenames in os.walk('../input/imagenetmini-1000/imagenet-mini/val/'):
#     for filename in filenames:
#         train_list.append(str(os.path.join(dirname, filename)))

# train_list = glob.glob('../input/imagenetmini-1000/imagenet-mini/train/**/*.JPEG')
# test_list = glob.glob('../input/imagenetmini-1000/imagenet-mini/val/**/*.JPEG')

In [ ]:
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(SEED)

In [ ]:
# gray_list = []
# for i in range(len(train_list)):
#     image = img.imread(train_list[i])
#     if(len(image.shape) == 2):
#         gray_list.append(train_list[i])
# for i in range(len(gray_list)):
#     train_list.remove(gray_list[i])
# print(len(gray_list))

# gray_list = []
# for i in range(len(test_list)):
#     image = img.imread(test_list[i])
#     if(len(image.shape) == 2):
#         gray_list.append(test_list[i])
# for i in range(len(gray_list)):
#     test_list.remove(gray_list[i])
# print(len(gray_list))
    
# gc.collect()

In [ ]:
# class MiniImageNetDataset(Dataset):
#     def __init__(self, x, y, transform=None):
#         self.list_x = x
#         self.list_y = y
#         self.transform = transform
    
#     def __len__(self):
#         return len(self.list_x)
    
#     def __getitem__(self, idx):
#         image = img.imread(self.list_x[idx])
#         label = self.list_y.index(self.list_x[idx].split('/')[5])
#         label = torch.as_tensor(label).to(device)
#         if self.transform is not None:
#             image = self.transform(image).to(device)
        
#         return {'image': image, 'label': label}

In [ ]:
# train_dataset = MiniImageNetDataset(train_list, label_list, train_transform)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
# for i, item in enumerate(train_dataloader):
#     plt.imshow(np.array(item['image'].cpu()).transpose([0, 2, 3, 1])[0])
#     plt.title(np.array(item['label'].cpu())[0])
#     plt.show()
#     break
# len(train_dataset)

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

#inception结构
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3reduce, ch3x3, ch5x5reduce, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3reduce, kernel_size=1),
            BasicConv2d(ch3x3reduce, ch3x3, kernel_size=3, padding=1)   # 保证输出大小等于输入大小
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5reduce, kernel_size=1),
            BasicConv2d(ch5x5reduce, ch5x5, kernel_size=5, padding=2)   # 保证输出大小等于输入大小
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

In [ ]:
#辅助分类器
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.averagePool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)  # output[batch, 128, 4, 4]

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = self.averagePool(x)
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x num_classes
        return x

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=True, init_weights=False):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
        self.lrn1 = nn.LocalResponseNorm(size=2)

        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.lrn2 = nn.LocalResponseNorm(size=2)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        x = self.lrn1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        x = self.lrn2(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        if self.training and self.aux_logits:   # eval model lose this layer
            return x, aux2, aux1
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize([224, 224]),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([224, 224]),
])

In [ ]:
# train_dataset = MiniImageNetDataset(train_list, label_list, train_transform)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
# test_dataset = MiniImageNetDataset(test_list, label_list, test_transform)
# test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=32)
train_dataset = torchvision.datasets.CIFAR100(root = './', train = True, transform = train_transform, download = True)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
test_dataset = torchvision.datasets.CIFAR100(root = './', train = False, transform = train_transform, download = True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32)

In [ ]:
net = GoogLeNet(num_classes=100, aux_logits=True, init_weights=True)
net.to(device)
# net.load_state_dict(torch.load('../input/googlenet-weights/googleNet.pth'))
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

best_acc = 0.0
save_path = './googleNet.pth'
for epoch in range(60):
    # train
    net.train()
    running_loss = 0.0
    for step, data in enumerate(train_dataloader):
        images, labels = data
        optimizer.zero_grad()
        logits, aux_logits2, aux_logits1 = net(images.to(device))
        loss0 = loss_function(logits, labels.to(device))
        loss1 = loss_function(aux_logits1, labels.to(device))
        loss2 = loss_function(aux_logits2, labels.to(device))
        loss = loss0 + loss1 * 0.3 + loss2 * 0.3
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # print train process
        rate = (step + 1) / len(train_dataloader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss), end="")
    print()

    # validate
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        for val_data in test_dataloader:
            val_images, val_labels = val_data
            outputs = net(val_images.to(device))
            predict_y = torch.max(outputs, dim=1)[1]
            acc += (predict_y == val_labels.to(device)).sum().item()
        val_accurate = acc / len(test_dataset)
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
        print('[epoch %d] train_loss: %.3f  test_accuracy: %.3f' %
              (epoch + 1, running_loss / step, val_accurate))

print('Finished Training')